<font color='darkblue'>
    
</font>

# Assignment 2

## Part 0: Building the network

<font color='darkblue'>
    To create our network, we downloaded the Zelda Wiki pages for all characters in BotW (during Week 4) and linked them via the hyperlinks connecting pages to each other. To achieve this goal we have used regular expressions!
</font>

* <font color='darkblue'>
    Explain the strategy you have used to extract the hyperlinks from the Wiki-pages, assuming that you have already collected the pages with the Zelda API.
</font>

* <font color='darkblue'>
    Show the regular expression(s) you have built and explain in details how it works.
</font>

## Part 1: Network visualization and basic stats

### Exercise 1a: Stats

* <font color='darkblue'>
    What is the number of nodes in the network?
</font>

* <font color='darkblue'>
    What is the number of links?
</font>

* <font color='darkblue'>
    Who is the top connected character in BotW? (Report results for the in-degrees and out-degrees). Comment on your findings. Is this what you would have expected?
</font>

* <font color='darkblue'>
    Who are the top 5 most connected allies (again in terms of in/out-degree)?
</font>

* <font color='darkblue'>
    Who are the top 5 most connected enemies -- bosses included -- (again in terms of in/out-degree)?
</font>

* <font color='darkblue'>
    Plot the in- and out-degree distributions.
</font>
* <font color='darkblue'>
    What do you observe?
</font>
* <font color='darkblue'>
    Can you explain why the in-degree distribution is different from the out-degree distribution?
</font>

* <font color='darkblue'>
    Find the exponent of the degree distribution (by using the powerlaw package) for the in- and out-degree distribution. What does it say about our network?
</font>

* <font color='darkblue'>
    Compare the degree distribution of the undirected graph to a random network with the same number of nodes and probability of connection p. Comment your results.
</font>

### Exercise 1b: Visualization

* <font color='darkblue'>
    Create a nice visualization of the total (undirected) network:
</font>
* <font color='darkblue'>
    Color nodes according to the role;
</font>
* <font color='darkblue'>
    Scale node-size according to degree;
</font>
* <font color='darkblue'>
    Get node positions based on the Force Atlas 2 algorithm;
</font>
* <font color='darkblue'>
    Whatever else you feel like that would make the visualization nicer.
</font>

* <font color='darkblue'>
    Describe the structure you observe. Can you identify nodes with a privileged position in the network? Do you observe chains of connected nodes? Do you see any interesting group of nodes (can you guess who's involved)?
</font>

## Part 2: Word-clouds

* <font color='darkblue'>
    Create a word-cloud for each race of the five champions of Hyrule (i.e. Hylian, Zora, Goron, Gerudo, and Rito) according to either TC-IDF. Feel free to make it as fancy as you like. Explain your process and comment on your results.
</font>

## Part 3: Communities and TF-IDF

<font color='darkblue'>
    Find communities and compute their associated TF-IDF
</font>

* <font color='darkblue'>
    Explain the Louvain algorithm and how it finds communities in a newtork.
</font>

* <font color='darkblue'>
    Explain how you chose to identify the communities: Which algorithm did you use? (if you did not use the Louvain method, explain how the method you have used works)
</font>

* <font color='darkblue'>
    Comment your results:
</font>
* <font color='darkblue'>
    How many communities did you find in total?
</font>
* <font color='darkblue'>
    Compute the value of modularity with the partition created by the algorithm.
</font>
* <font color='darkblue'>
    Plot the distribution of community sizes.
</font>

* <font color='darkblue'>
    For the 5 largest communities, create TF-IDF based rankings of words in each community.
</font>
* <font color='darkblue'>
    There are many ways to calculate TF-IDF, explain how you've done it and motivate your choices.
</font>
* <font color='darkblue'>
    List the 5 top words for each community according to TF.
</font>
* <font color='darkblue'>
    List the 5 top words for each community accourding to TF-IDF. Are these words more descriptive of the community than just the TF? Justify your answer.
</font>

## Part 4: Sentiment of communities

<font color='darkblue'>
    Analyze the sentiment of the communities (lecture 8). Here, we assume that you've successfully identified communities.
</font>
<font color='darkblue'>
    A couple of additional instructions you will need below:
</font>
* <font color='darkblue'>
    We name each community by its three most connected characters.
</font>
* <font color='darkblue'>
    Average the average sentiment of the nodes in each community to find a community level sentiment.
</font>

<font color='darkblue'>
    Here's what you need to do (repeat these steps and report your results for both LabMT and VADER):
</font>

* <font color='darkblue'>
    Calculate and store sentiment for every character
</font>

* <font color='darkblue'>
    Create a histogram of all character's associated sentiments.
</font>

* <font color='darkblue'>
    What are the 10 characters with happiest and saddest pages?
</font>

<font color='darkblue'>
    Now, compute the sentiment of communities:
</font>

* <font color='darkblue'>
    What are the three happiest communities according to the LabMT wordlist approach? What about VADER?
</font>

* <font color='darkblue'>
    What are the three saddest communities according to the LabMT wordlist approach? What about VADER?
</font>

* <font color='darkblue'>
    Create a bar plot showing the average sentiment of each community and add error-bars using the standard deviation for both methods.
</font>

* <font color='darkblue'>
    Explain the difference between the two methods and compare the results you have obtained above.
</font>

* <font color='darkblue'>
    What is the advantage of using a rule-based method over the dictionary-based approach?
</font>